<a href="https://colab.research.google.com/github/Jarvis-BITS/MIT-midodrine/blob/main/Midodrine_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

Tensorflow version 2.8.0
Authenticated


run if TPU's are *connected*

In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

run if GPU's connected

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
from google.colab import auth

auth.authenticate_user()
print('Authenticated')
 

Authenticated


# Demographic analysis


---


**List of variables/column names we should pull**: 

1.   from mimic_core.admissions: subject_id, gender, admittime,	dischtime, deathtime, ethnicity
2.   from derived: weight/height, age, some indicators related to midodrine injection (still in search)




In [2]:
%load_ext google.colab.data_table

In [10]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [ ]:
bqclient = bigquery.Client(project='my-project-eicu')

In [169]:
# Population Extraction
Population = """
WITH Demographic AS (
  SELECT subject_id, intime, outtime, los, first_careunit
  FROM `physionet-data.mimic_icu.icustays`
  WHERE first_careunit = last_careunit
)
, Age AS (
  SELECT subject_id, anchor_age, gender
  FROM `physionet-data.mimic_core.patients`
)
, Midodrine_injection AS(
  SELECT subject_id, starttime
  FROM `physionet-data.mimic_hosp.pharmacy`
  WHERE lower(medication) like 'midodrine'
)
, exclude_Midodrine_date AS(
  SELECT subject_id
  , CASE WHEN LOGICAL_OR(starttime < DATE_TRUNC(intime - interval '1' day, DAY) or starttime > outtime) THEN 1 ELSE 0 END AS exclude_midodrine
  FROM `physionet-data.mimic_icu.icustays` LEFT JOIN Midodrine_injection USING (subject_id)
  GROUP BY subject_id
)
, Midodrine AS (
  SELECT subject_id
  , CASE WHEN LOGICAL_OR(medication = 'midodrine' or medication = 'Midodrine') THEN 1 ELSE 0 END AS midodrine_injection
  , AVG(doses_per_24_hrs) AS avg_doses_per_24_hrs
  FROM `physionet-data.mimic_hosp.pharmacy`  
  WHERE doses_per_24_hrs is not null 
  GROUP BY subject_id
)
, Population AS (
  SELECT *
  FROM(SELECT DISTINCT subject_id FROM `physionet-data.mimic_icu.icustays`)
  LEFT JOIN Demographic USING (subject_id)
  LEFT JOIN Midodrine USING (subject_id)
  LEFT JOIN exclude_Midodrine_date USING (subject_id)
  LEFT JOIN Age USING (subject_id)
)
SELECT * FROM Population
"""

In [170]:
population_data = (
    bqclient.query(Population)
    .result()
    .to_dataframe()
)

In [172]:
population_data

,subject_id,intime,outtime,los,first_careunit,midodrine_injection,avg_doses_per_24_hrs,exclude_midodrine,anchor_age,gender
0,14303051,2175-11-20 01:02:00,2175-11-21 16:06:17,1.627975,Neuro Stepdown,0.0,2.090909,0,24,F
1,13386137,2125-09-27 20:05:47,2125-09-28 16:32:59,0.852222,Neuro Stepdown,0.0,1.540541,0,20,F
2,13315365,2178-03-03 20:16:03,2178-03-04 16:46:26,0.854433,Neuro Stepdown,0.0,1.600000,0,37,F
3,18917282,2153-11-01 12:06:00,2153-11-01 15:40:42,0.149097,Neuro Stepdown,0.0,1.800000,0,23,M
4,10318816,2184-08-12 06:31:00,2184-08-13 15:52:40,1.390046,Neuro Stepdown,0.0,2.000000,0,29,F
...,...,...,...,...,...,...,...,...,...,...
73719,18227829,2177-11-18 13:20:00,2177-11-19 19:36:10,1.261227,Medical/Surgical Intensive Care Unit (MICU/SICU),0.0,1.354839,0,91,M
73720,13762560,2144-11-24 02:45:00,2144-11-24 22:28:50,0.822106,Medical/Surgical Intensive Care Unit (MICU/SICU),0.0,1.000000,0,91,F
73721,14247952,2139-09-16 14:40:00,2139-09-19 18:18:37,3.151817,Medical/Surgical Intensive Care Unit (MICU/SICU),0.0,1.000000,0,91,F
73722,15694615,2111-09-13 04:30:31,2111-09-13 14:58:22,0.436007,Medical/Surgical Intensive Care Unit (MICU/SICU),0.0,1.428571,0,91,M


## Next, we need to build the cohort table
### First, exclude the child (< 18 years' old)
### Second, exclude those who have been to ICU more than once
### Third, exclude those whose ICU care is not 'MICU' or 'SICU'
### Fourth, exclude those who accepted midodrine more than one day before icu care or after icu care (have been done before)

In [174]:
# first part
cohort_first = population_data[population_data['anchor_age']>18]
cohort_first.shape

(73618, 10)

In [176]:
# second part
cohort_second = cohort_first.drop_duplicates(subset='subject_id', 
                                             keep=False, 
                                             inplace=False,
                                             ignore_index=False)
cohort_second.shape

(41245, 10)

In [180]:
# Third
cohort_third = cohort_second[(cohort_second['first_careunit'].str.contains('MICU')) | (cohort_second['first_careunit'].str.contains('SICU'))]
cohort_third.shape

(25093, 10)

In [181]:
# Fourth
cohort_final = cohort_third[cohort_third['exclude_midodrine']==0]
cohort_final.shape

(24581, 10)